In [1]:
import pandas as pd
import numpy as np
np.warnings.filterwarnings('ignore')
import xarray as xr
from metpy.units import units
from metpy.plots import SkewT
import metpy.calc as mpcalc
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('/home/franzihe/Documents/Python/Thesis/')
import createFolder as cF


In [2]:
# plot cosmetics
sns.set_context('paper', font_scale=1.6)

sns.set(font = 'Serif', font_scale = 1.6, )
sns.set_style('ticks', 
                  {'font.family':'serif', #'font.serif':'Helvetica'
                   'grid.linestyle': '--',
                   'axes.grid': True,
                  }, 
                   )
# Set the palette to the "pastel" default palette:
sns.set_palette("colorblind")

In [3]:
savefig = 1
if savefig == 1:
    figdir = '/home/franzihe/Documents/Figures/Weathermast_MEPS_Retrieval/Haukeliseter/MEPS_CTRL_ICET/' 
    cF.createFolder('%s/' %figdir)
    form = 'png'

In [4]:


hour = '12'

m = ['12','01', '02']
h = ['00', '12']
meps_run = [ 'CTRL', 'ICE-T', ]

In [5]:
# Select col_names to be importet for the sounding plot
col_names = ['PRES', 'HGHT', 'TEMP', 'DWPT', 'MIXR', 'DRCT', 'SKNT', 'THTA']
header = np.arange(0,6)

In [6]:
def concat_profile_all_days(df, Date, observation, _pres, _temp, _dwpt, _xwind, _ywind):
    _lev = np.arange(1000,-25, -25)
    _averaged = pd.DataFrame()

    for i in _lev:
        filter1 = np.logical_and(df.PRES > i-25,
                                 df.PRES <= i+25 ) 
                
        _averaged = pd.concat([_averaged, df.where(filter1).mean()], axis = 1)
        _averaged = _averaged.rename(columns = {0:i})

        _averaged = _averaged.T
            
            # concat the pressure, height, temperature, dewpoint, mixing ration, wind direction, wind speed, 
            # potential temperature of all dates 
        _pres = pd.concat([_pres, _averaged.PRES], axis = 1).rename(columns = {'PRES':Date})
#            _hght = pd.concat([_hght, _averaged.HGHT], axis = 1).rename(columns = {'HGHT':Date})
        _temp = pd.concat([_temp, _averaged.TEMP], axis = 1).rename(columns = {'TEMP':Date})
        _dwpt = pd.concat([_dwpt, _averaged.DWPT], axis = 1).rename(columns = {'DWPT':Date})
  #          _mixr = pd.concat([_mixr, _averaged.MIXR], axis = 1).rename(columns = {'MIXR':Date})
   #         _drct = pd.concat([_drct, _averaged.DRCT], axis = 1).rename(columns = {'DRCT':Date})
    #        _sknt = pd.concat([_sknt, _averaged.SKNT], axis = 1).rename(columns = {'SKNT':Date})
     #       _thta = pd.concat([_thta, _averaged.THTA], axis = 1).rename(columns = {'THTA':Date})
        _xwind = pd.concat([_xwind, _averaged.x_wind], axis = 1).rename(columns = {'x_wind':Date})
        _ywind = pd.concat([_ywind, _averaged.y_wind], axis = 1).rename(columns = {'y_wind':Date})
            
    return(_pres, _temp, _dwpt, _xwind, _ywind)

In [7]:
p = dict()
T = dict()
Td = dict()
u = dict()
v = dict()
p_meps = dict()
T_meps = dict()
Td_meps = dict()
u_meps = dict()
v_meps = dict()

In [8]:
for hour in h:
    
    _temp = pd.DataFrame()
    _pres = pd.DataFrame()
    _hght = pd.DataFrame()
    _temp = pd.DataFrame()
    _dwpt = pd.DataFrame()
    _mixr = pd.DataFrame()
    _drct = pd.DataFrame()
    _sknt = pd.DataFrame()
    _thta = pd.DataFrame()
    _xwind = pd.DataFrame()
    _ywind = pd.DataFrame()

    _pres_meps = pd.DataFrame()
    _temp_meps = pd.DataFrame()
    _dwpt_meps = pd.DataFrame()
    _xwind_meps = pd.DataFrame()
    _ywind_meps = pd.DataFrame()
    for month in m:
        if month == '12':
            t = np.array([8, 9, 10, 12, 15, 20, 21, 22, 23, 24, 25, 26, 29, 31])
        if month == '01':
            t = np.array([2, 3, 5, 6, 8, 9, 10, 11, 12, 28])
        if month == '02':
            t = np.array([2, 3, 4])    
        if month == '12':
            year = '2016'
        if month == '01' or month == '02':
            year = '2017'
        for day in t:
            if day < 10:
                day = '0%s' %day
            Date = year+month+str(day)

            stn = '01415' #1415 is ID for Stavanger
            Sounding_filename = '/home/franzihe/Documents/Data/Sounding/{}/{}{}{}_{}.txt'.format(stn,year,month,str(day),hour)


            df = pd.read_table(Sounding_filename, delim_whitespace=True, skiprows = header, \
                               usecols=[0, 1, 2, 3, 5, 6, 7, 8], names=col_names)

            ### the footer changes depending on how high the sound measured --> lines change from Radiosonde to Radiosonde
            # 1. find idx of first value matching the name 'Station'
            lines = df.index[df['PRES'].str.match('Station')]
            if len(lines) == 0:
                print('no file found: %s%s%s_%s' %(year,month,day,hour))
            #    continue
            else:
                # read in the Sounding files
                idx = lines[0]
                footer = np.arange((idx+header.size),220)
                skiprow = np.append(header,footer)
                df = pd.read_table(Sounding_filename, delim_whitespace=True,  skiprows = skiprow, \
                               usecols=[0, 1, 2, 3, 5, 6, 7, 8], names=col_names)
                df['x_wind'], df['y_wind'] = mpcalc.wind_components(df.SKNT.values *units.knots, df.DRCT.values*units.degrees)



    #            _lev = np.arange(1000,-25, -25)
     #           _averaged = pd.DataFrame()

      #          for i in _lev:
       #             filter1 = np.logical_and(df.PRES > i-25,
        #                                     df.PRES <= i+25 ) 
         #           
          #          _averaged = pd.concat([_averaged, df.where(filter1).mean()], axis = 1)
           #         _averaged = _averaged.rename(columns = {0:i})
    #
     #           _averaged = _averaged.T

                # concat the pressure, height, temperature, dewpoint, mixing ration, wind direction, wind speed, 
                # potential temperature of all dates 
      #          _pres = pd.concat([_pres, _averaged.PRES], axis = 1).rename(columns = {'PRES':Date})
       #         _hght = pd.concat([_hght, _averaged.HGHT], axis = 1).rename(columns = {'HGHT':Date})
        #        _temp = pd.concat([_temp, _averaged.TEMP], axis = 1).rename(columns = {'TEMP':Date})
         #       _dwpt = pd.concat([_dwpt, _averaged.DWPT], axis = 1).rename(columns = {'DWPT':Date})
          #      _mixr = pd.concat([_mixr, _averaged.MIXR], axis = 1).rename(columns = {'MIXR':Date})
           #     _drct = pd.concat([_drct, _averaged.DRCT], axis = 1).rename(columns = {'DRCT':Date})
            #    _sknt = pd.concat([_sknt, _averaged.SKNT], axis = 1).rename(columns = {'SKNT':Date})
             #   _thta = pd.concat([_thta, _averaged.THTA], axis = 1).rename(columns = {'THTA':Date})
              #  _xwind = pd.concat([_xwind, _averaged.x_wind], axis = 1).rename(columns = {'x_wind':Date})
               # _ywind = pd.concat([_ywind, _averaged.y_wind], axis = 1).rename(columns = {'y_wind':Date})
                _pres, _temp, _dwpt, _xwind, _ywind = concat_profile_all_days(df, Date, 'RS', _pres, _temp, _dwpt, _xwind, _ywind)



                # read in the MEPS runs
    #            for meps in meps_run:
                meps = 'CTRL'
                stn = 'Stavanger'
                meps_dirnc = '/home/franzihe/Documents/Data/MEPS/%s/%s/%s_00.nc' %(stn,meps,Date)
                meps_f = xr.open_dataset(meps_dirnc, drop_variables ={'air_temperature_0m','liquid_water_content_of_surface_snow','rainfall_amount', 'snowfall_amount', 'graupelfall_amount', 'surface_air_pressure', 'surface_geopotential',
                                                          'precipitation_amount_acc', 'integral_of_snowfall_amount_wrt_time', 'integral_of_rainfall_amount_wrt_time',
                                                         'integral_of_graupelfall_amount_wrt_time', 'surface_snow_sublimation_amount_acc', 'air_temperature_2m','relative_humidity_2m',
                                                         'specific_humidity_2m', 'x_wind_10m', 'y_wind_10m', 'air_pressure_at_sea_level', 
                                                         'atmosphere_cloud_condensed_water_content_ml', 'atmosphere_cloud_ice_content_ml', 'atmosphere_cloud_snow_content_ml','atmosphere_cloud_rain_content_ml', 'atmosphere_cloud_graupel_content_ml',
                                                         'pressure_departure', 'layer_thickness', 'geop_layer_thickness'},
                                        ).reset_index(dims_or_levels = ['height0', 'height1', 'height3', 'height_above_msl', ], drop=True).sortby('hybrid', ascending = False)
                # pressuer into hPa
                meps_f['pressure_ml'] = meps_f.pressure_ml/100
                # air temperature has to be flipped, something was wrong when reading the data from Stavanger
                meps_f['air_temperature_ml'] = (('time', 'hybrid',),meps_f.air_temperature_ml.values[:,::-1] - 273.15)
                meps_f['specific_humidity_ml'] = (('time', 'hybrid',),meps_f.specific_humidity_ml.values[:,::-1])
                meps_f['x_wind_ml'] = (('time', 'hybrid',), meps_f.x_wind_ml.values[:,::-1])
                meps_f['y_wind_ml'] = (('time', 'hybrid',), meps_f.y_wind_ml.values[:,::-1])

                # calculate the dewpoint by first calculating the relative humidity from the specific humidity
                meps_f['relative_humidity'] = (('time', 'hybrid', ), mpcalc.relative_humidity_from_specific_humidity(meps_f.pressure_ml.values * units.hPa, 
                                                                                  meps_f.air_temperature_ml.values * units.degC, 
                                                                                  meps_f.specific_humidity_ml.values * units('kg/kg')))

                meps_f['DWPT'] = (('time', 'hybrid',), mpcalc.dewpoint_from_relative_humidity(meps_f.air_temperature_ml.values * units.degC, meps_f.relative_humidity))

                if hour == '12':
                    meps_f = meps_f.isel(time = 11).to_dataframe()
                elif hour == '00':
                    meps_f = meps_f.isel(time = 23).to_dataframe()

                meps_f = meps_f.rename(columns = {'x_wind_ml':'x_wind', 'y_wind_ml':'y_wind', 'pressure_ml':'PRES', 'air_temperature_ml':'TEMP'})

                _pres_meps, _temp_meps, _dwpt_meps, _xwind_meps, _ywind_meps = concat_profile_all_days(meps_f, Date, 'MEPS',
                                                                                                      _pres_meps, _temp_meps, _dwpt_meps, _xwind_meps, _ywind_meps)




    ## average pressure, height, temperature, dewpoint, mixing ration, wind direction, wind speed, 
    # potential temperature over time to get seasonal mean and assign units.
    p[hour] = _pres.mean(axis = 1, skipna=True).values * units.hPa
    #z = _hght.mean(axis = 1, skipna=True).values * units.meter
    T[hour] = _temp.mean(axis = 1, skipna=True).values * units.degC
    Td[hour] = _dwpt.mean(axis = 1, skipna=True).values * units.degC
    #qv = _mixr.mean(axis = 1, skipna=True).values * units('g/kg')
    #WD = _drct.mean(axis = 1, skipna=True).values * units.degrees
    #WS = _sknt.mean(axis = 1, skipna=True).values * units.knots
    #th = _thta.mean(axis = 1, skipna=True).values * units.kelvin
    u[hour] = _xwind.mean(axis = 1, skipna = True)
    v[hour] = _ywind.mean(axis = 1, skipna = True)

    p_meps[hour] = _pres_meps.mean(axis = 1, skipna=True).values * units.hPa
    T_meps[hour] = _temp_meps.mean(axis = 1, skipna=True).values * units.degC
    Td_meps[hour] = _dwpt_meps.mean(axis = 1, skipna=True).values * units.degC
    u_meps[hour] = _xwind_meps.mean(axis = 1, skipna = True)
    v_meps[hour] = _ywind_meps.mean(axis = 1, skipna = True)

    #u, v = mpcalc.wind_components(WS, WD)

no file found: 20161224_00
no file found: 20170102_00
no file found: 20170103_00
no file found: 20170112_00
no file found: 20170128_00
no file found: 20170202_00
no file found: 20170203_00
no file found: 20161223_12
no file found: 20161224_12
no file found: 20170102_12
no file found: 20170103_12
no file found: 20170128_12
no file found: 20170202_12


In [9]:
def plt_skewT(skew, p, T, p_meps, T_meps, Td, Td_meps, u, v, profile_time):
    # Plot the data using normal plotting functions, in this case using
    # log scaling in Y, as dictated by the typical meteorological plot
    skew.plot(p, T, )
    skew.plot(p_meps, T_meps, )

    skew.plot(p, Td, )
    skew.plot(p_meps, Td_meps, )

    skew.plot_barbs(p, u, v)
    skew.ax.set_ylim(1000, 100)

    # Add the relevant special lines
    skew.plot_dry_adiabats()
    skew.plot_moist_adiabats()
    skew.plot_mixing_lines()

    # Good bounds for aspect ratio
    skew.ax.set_xlim(-30, 40)
    skew.ax.text(0.05, 1, 'Vertical profile mean - Stavanger: {} UTC'.format(profile_time), transform=skew.ax.transAxes,
                 fontsize=14, verticalalignment='bottom',)# bbox='fancy')

In [10]:
fig = plt.figure(figsize=(18, 9))

#plot skewT for 00UTC 
skew = SkewT(fig, rotation=45,subplot=121)
plt_skewT(skew, p['00'], T['00'], p_meps['00'], T_meps['00'], Td['00'], Td_meps['00'], u['00'], v['00'], '00')

skew = SkewT(fig, rotation=45,subplot=122)
plt_skewT(skew, p['12'], T['12'], p_meps['12'], T_meps['12'], Td['12'], Td_meps['12'], u['12'], v['12'], '12')

if savefig == 1:
        cF.createFolder('%s/' %(figdir))
        fig_name = 'winter_16_17_vertical_profile.'+form
        plt.savefig('%s/%s' %(figdir, fig_name), format = form, bbox_inches='tight', transparent=True)
        print('plot saved: %s/%s' %(figdir, fig_name))
        plt.close()

plot saved: /home/franzihe/Documents/Figures/Weathermast_MEPS_Retrieval/Haukeliseter/MEPS_CTRL_ICET//winter_16_17_vertical_profile.png
